# Query Expansion

0) Just some imports

In [1]:
import re
import math
import numpy as np
import common as cm

# 1) Simple search engine

1.1) Get acquainted with the below class. There are several TODOs. However, DO NOT complete them now. 

In [2]:
class Dictionary:
    def __init__(self):
        ### keeps unique terms (SORTED!)
        self.terms = self.loadTerms("terms.txt")
        self.idfs = [] ### IDF coefficients
        self.corM = [] ### a correlation matrix

    ### load terms
    def loadTerms(self, fileName):
        file = open(fileName,'r', encoding='utf-8-sig')
        k = [self.proc(s) for s in file.readlines()]
        k.sort()
        return k

    ### ignore it
    def proc(self, s):
        if s[-1] == '\n': return s[:-1]
        else: return s
    
    ### TODO (DO NOT FINISH THIS METHOD YET. YOU WILL BE ASKED FOR IT LATER) 
    def computeIDFs(self, documents):
        n = len(documents)
        for term in self.terms:
            counter = 0
            for d in documents:
                if term in d.tokens:
                    counter += 1
            self.idfs.append(np.log(n/counter))
    
    ### TODO (DO NOT FINISH THIS METHOD YET. YOU WILL BE ASKED FOR IT LATER) 
    def computeCorM(self, documents):
        for d1 in documents:
            row = []
            for d2 in documents:
                z = d1.wordsSet.intersection(d2.wordsSet)
                y = d1.wordsSet.union(d2.wordsSet)
                if d1 == d2:
                    row.append(-1)
                else:
                    print(len(d1.wordsSet))
                    print(len(d2.wordsSet))
                    row.append(len(z)/len(y))
            self.corM.append(row)
        

### SOME DEBUG
dictionary = Dictionary()
print(dictionary.terms[:50])

['aaai', 'about', 'academic', 'access', 'acquired', 'acquisition', 'action', 'activity', 'actual', 'adaptive', 'add', 'advance', 'agricultural', 'aha', 'aim', 'alert', 'algorithm', 'all', 'analysis', 'and', 'announcement', 'answer', 'anyone', 'application', 'applied', 'apply', 'applying', 'approach', 'approache', 'april', 'archive', 'are', 'area', 'areas', 'article', 'artificial', 'asked', 'august', 'author', 'automated', 'automatically', 'autonomous', 'available', 'awards', 'backend', 'backgammon', 'baldi', 'based', 'basic', 'bayesian']


1.2) Load files: here we load some example collection of documents. RAW_DOCUMENTS = just strings. Check if the documents are loaded correctly (e.g., print RAW_DOCUMENTS[0])

In [3]:
RAW_DOCUMENTS = cm.loadDocuments("documents.txt")
### SOME DEBUG
print(RAW_DOCUMENTS[0])

David W. Aha:  Machine Learning Page
 Machine Learning Resources. Suggestions welcome. ... (WizRule); ZDM Scientific
 Ltd. Conference Announcements. Courses on Machine Learning. Data Repositories. ... 
 Description: Comprehensive machine learning resources from Applications to Tutorials.



In [4]:
### SOME DEBUG, JUST RUN; check if (a) common.py is imported correctly and (b) 
### tokens are correctly derived from some document (e.g., RAW_DOCUMENTS[0])
print(cm.simpleTextProcessing(RAW_DOCUMENTS[0], re))

['david', 'aha', 'machine', 'learning', 'page', 'machine', 'learning', 'resource', 'suggestion', 'welcome', 'wizrule', 'zdm', 'scientific', 'ltd', 'conference', 'announcement', 'course', 'machine', 'learning', 'data', 'repository', 'description', 'comprehensive', 'machine', 'learning', 'resource', 'from', 'application', 'tutorials']


1.3) Get acquainted with the below class. 

In [13]:
class Document:
    def __init__(self, doc_id, raw_document, dictionary):
        self.doc_id = doc_id ### DOC ID, simply 0,1,2,3....
        self.raw_document = raw_document ### raw data, i.e., string data
        self.dictionary = dictionary # reference to the dictionary
        
        ### DOCUMENT REPRESENTATIONS
        self.tokens = cm.simpleTextProcessing(raw_document, re) ### get terms
        self.bow = [] # Bag Of Words (BOW - number of term occurences)
        self.words = []
        self.wordsSet = set()
        self.tf = [] # TF representation
        self.tf_idf = [] # TF-IDF representation

    ### TODO - complete this method; it should compute a BOW representation
    def computeBOW_Representation(self):
        self.bow = []
        self.wordsSet = set(self.tokens)
        self.words = list(self.wordsSet)
        for w in self.words:
            self.bow.append([w, self.tokens.count(w)])
    
    ### TODO - complete this method; it should compute a TF representation
    def computeTF_Representation(self):
        self.tf = []
        m = 0
        for pair in self.bow:
            if pair[1] > m:
                m = pair[1]
        for pair in self.bow:
            self.tf.append(pair[1]/m)
    
    ### TODO - complete this method; it should compute a TFxIDF representation 
    ### (important: it should not be run before dictionary.idfs are not computed!)
    def computeTF_IDF_Representation(self):
        self.tf_idf = [] 
        for pair in self.bow:
            # index w idf
            if pair[0] in self.dictionary.terms:
                termIndex = dictionary.terms.index(pair[0])
                # wartość w idf
                idfValue = dictionary.idfs[termIndex]
            else:
                idfValue = 0
            # index w tf
            tfIndex = self.bow.index(pair)
            # wartość w tf
            tfValue = self.tf[tfIndex]
            
            self.tf_idf.append(idfValue * tfValue)
    
    def computeRepresentations(self):
        self.computeBOW_Representation()
        self.computeTF_Representation()
        self.computeTF_IDF_Representation()
    
documents = [Document(i, RAW_DOCUMENTS[i], dictionary) for i in range(len(RAW_DOCUMENTS))]


1.4) Compute IDFs here

In [14]:
### TODO COMPUTE IDFS HERE (FINISH THE PROPER METHOD OF THE DICTIONARY CLASS - DO NOT FORGET TO RE-RUN THE CELL)
dictionary.computeIDFs(documents)

### SOME DEBUG
res = [[dictionary.terms[i], dictionary.idfs[i]] for i in range(len(dictionary.terms))]
res.sort(key = lambda x: x[1])
# LEAST COMMON WORDS - HIGH IDF
print(res[-5:])
# MOST COMMON WORDS - LOW IDF
print(res[:5])

[['working', 4.477336814478207], ['www', 4.477336814478207], ['york', 4.477336814478207], ['young', 4.477336814478207], ['zdm', 4.477336814478207]]
[['learning', 0.02298951822469878], ['machine', 0.02298951822469878], ['the', 0.4519851237430572], ['and', 0.7396671961948382], ['description', 0.8938178760220965]]


1.5) Compute the document representations (for each document run computeRepresentations())

In [15]:
for d in documents: d.computeRepresentations()
### SOME DEBUG (you should see some 4s - which terms are these?)
print(documents[0].bow)

[['suggestion', 1], ['course', 1], ['resource', 2], ['page', 1], ['wizrule', 1], ['aha', 1], ['announcement', 1], ['scientific', 1], ['application', 1], ['david', 1], ['data', 1], ['description', 1], ['comprehensive', 1], ['learning', 4], ['conference', 1], ['machine', 4], ['ltd', 1], ['from', 1], ['tutorials', 1], ['welcome', 1], ['repository', 1], ['zdm', 1]]


1.6) Finish the below method. It should compute and return a cosine similarity (v1 and v2 are two vectors - tf-idf representations)

In [16]:
### TODO 
def getSimilarity(v1, v2):
    listOfSums = []
    for term in dictionary.terms:
        if term in v1.words:
            tfIdfIndex = v1.words.index(term)
            v1Value = v1.tf_idf[tfIdfIndex]
        else:
            v1Value = 0
        if term in v2.words:
            tfIdfIndex = v2.words.index(term)
            v2Value = v2.tf_idf[tfIdfIndex]
        else:
            v2Value = 0
        listOfSums.append(v1Value*v2Value)
    nominator = sum(listOfSums)
    sumV1 = 0
    for tf_idf in v1.tf_idf:
        sumV1 += tf_idf ** 2
    sumV2 = 0
    for tf_idf in v2.tf_idf:
        sumV2 += tf_idf ** 2
    denominator = (sumV1 * sumV2) ** (1/2)
    return nominator/denominator
        
print(getSimilarity(documents[0], documents[1]))

0.08769544271231881


1.7) Run the below script for different queries. getTopResults seeks for documents being the most similar/relevant to the query. Do you find the results satisfactory?

In [20]:
query = "machine learning"
# query = "academic research"
# query = "international conference"
# query = "international conference washington"

In [21]:
def getTopResults(query, documents, dictionary, similarity, top = 5):
    qd = Document(-1, query, dictionary)
    qd.computeRepresentations()
    ranks = [[d, getSimilarity(d, qd)] for d in documents]
    ranks.sort(key=lambda x: -x[1])
    for i in range(top):
        print("RANK = " + str(i+1) + " WITH SIMILARITY = " + str(ranks[i][1]) + " | DOC ID = " + str(ranks[i][0].doc_id))
        print(ranks[i][0].raw_document)
        print("")

getTopResults(query, documents, dictionary, getSimilarity, top = 5)

RANK = 1 WITH SIMILARITY = 0.017145022655810024 | DOC ID = 63
AI / Machine Learning Resources
 AI / Machine Learning Resources. General Machine Learning. The Journal
 of Machine Learning. MLnet Machine Learning Archive at GMD. The ... 


RANK = 2 WITH SIMILARITY = 0.012801913843626708 | DOC ID = 77
Machine Learning
 Machine Learning. Machine Learning Home Page (Editor) Machine Learning Home
 Page (Publisher) Machine Learning Online by Kluwer Academic Publishers: ... 


RANK = 3 WITH SIMILARITY = 0.012340729913628307 | DOC ID = 34
Machine Learning
 Machine Learning. Related Sites. Machine Learning Resources courtesy
 of David Aha A Machine Learning Tutorial a good overview of the ... 


RANK = 4 WITH SIMILARITY = 0.0110699100227987 | DOC ID = 81
Oxford University Machine Learning Group
 Machine Learning at the Computing Laboratory. ... Logic Programming and
 Learning and Intelligent Systems. Other Machine Learning Groups. ... 


RANK = 5 WITH SIMILARITY = 0.010455352603751217 | DOC ID =

# 2) Query expansion

## 2.1) Correlation matrix

2.1.1) Finish dictionary.computeCorM method (see class Dictionary). It should generate a correlation matrix (correlation between terms).

IMPORTANT: although corM[ i ][ i ] (for each i) should be 1.0, set it to -1.0

In [22]:
### TODO - COMPLETE THE computeCorM METHOD (see one of the first cells)
dictionary.computeCorM(documents)
print(dictionary.corM)

10714285714285714, 0.08571428571428572, 0.10714285714285714, 0.14814814814814814, 0.06896551724137931, 0.05555555555555555, 0.16, 0.1111111111111111, 0.14814814814814814, 0.08, 0.16666666666666666, 0.0967741935483871, 0.13043478260869565, 0.09090909090909091, 0.1111111111111111, 0.14705882352941177, 0.10344827586206896, 0.07142857142857142, 0.09090909090909091, 0.07692307692307693, 0.13043478260869565, 0.07692307692307693, 0.0625, 0.12, 0.16, 0.20833333333333334, 0.08333333333333333, 0.08, 0.06666666666666667, 0.17391304347826086, 0.17857142857142858, 0.09523809523809523, 0.16, 0.1111111111111111, 0.07692307692307693, 0.10344827586206896, 0.10526315789473684, 0.16666666666666666, 0.15384615384615385, 0.125, 0.08571428571428572, 0.18181818181818182, 0.18518518518518517, 0.1282051282051282, 0.20588235294117646, 0.12, 0.125, 0.1, 0.13043478260869565, 0.08333333333333333, 0.17391304347826086, 0.25, 0.1111111111111111, 0.09090909090909091, 0.07407407407407407, 0.09523809523809523, 0.0588235

2.1.2) Finish the below method. For each term in the query (you must parse the query, see getTopResults() method), find another term which is the most correlated with the input term.

In [24]:
# query = "machine"
# query = "algorithm"
# query = "learning"
query = "conference"
# query = "research"
# query = "concept"

def suggestKeywords(query, dictionary):
    ### TODO
    matrix = []
    for term in dictionary.terms:
        row = []
        suma = 0
        for d in documents:
            if term in d.wordsSet:
                idx = d.words.index(term)
                row.append(d.bow[idx][1])
                suma+= d.bow[idx][1] ** 2
            else:
                row.append(0)
        row = [i / (suma ** (1 / 2)) for i in row]
        matrix.append(row)
    matrix = np.asarray(matrix)
    finalM = matrix@matrix.T
    idx = dictionary.terms.index(query)
    for i in range(len(finalM)):
        finalM[i, i] *= -1
    m = max(finalM[idx])
    xd = 0
    for i in range(len(finalM[idx])):
        if finalM[idx, i] == m:
            xd = i
    
    print(dictionary.terms[xd])
        
suggestKeywords(query, dictionary)

international


# 2.2) Rocchio algorithm

$\overrightarrow{q_{m}} = \alpha \overrightarrow{q} + \left(\beta \cdot \dfrac{1}{|D_{r}|} \sum_{\overrightarrow{D_j} \in D_{r}} \overrightarrow{D_j} \right) - \left(\gamma \cdot \dfrac{1}{|D_{nr}|} \sum_{\overrightarrow{D_j} \in D_{nr}} \overrightarrow{D_j} \right)$

2.2.1) Firstly, run the below code. Observe the results. Assume that we do not like the first and the second result (Docs 63 and 77). However, assume that docs 29 and 36 are satisfactory. Now, modfify the method. It should alter the query vector, according to Rocchio algorithm. Check the result for the above considered scenario (relevant docs = 29 and 36; not relevant = 63 and 77). Check the results for different values of alpha, beta, and gamma coefficients. 

In [25]:
def getTopResults_Rocchio(query, 
                          documents, 
                          dictionary, 
                          similarity, 
                          rel_docs = [29, 36],
                          nrel_docs = [63, 77],
                          alpha = 0.5,
                          beta = 0.3,
                          gamma = 0.2,
                          top = 10):
    qd = Document(-1, query, dictionary)
    qd.computeRepresentations()
    ##### TODO: MODIFY qd.tf_idf HERE
    
    A = [alpha * i for i in qd.tf_idf]
    print(len(documents[29].tf_idf))
    bSum = [0 for i in documents[0].tf_idf]
    print(len(bSum))
    for i in rel_docs:
        for b in range(len(bSum)):
            bSum[b] += documents[i].tf_idf[b]
    print(bSum)

    #####
    ranks = [[d, getSimilarity(d.tf_idf, qd.tf_idf)] for d in documents]
    ranks.sort(key=lambda x: -x[1])
    for i in range(top):
        print("RANK = " + str(i+1) + " WITH SIMILARITY = " + str(ranks[i][1]) + " | DOC ID = " + str(ranks[i][0].doc_id))
        print(ranks[i][0].raw_document)
        print("")

getTopResults_Rocchio("machine learning", documents, dictionary, getSimilarity, top = 10)

14
22


IndexError: list index out of range

# 2.3) WordNet

2.3.1) Installation

http://www.nltk.org/install.html

import nltk 

nltk.download()

https://www.nltk.org/data.html

In [62]:
from nltk.corpus import wordnet as wn
import nltk

ntlk.download()

NameError: name 'ntlk' is not defined

Definition: synset = (from wiki) (information science) A set of one or more synonyms that are interchangeable in some context without changing the truth value of the proposition in which they are embedded.

2.3.2) Display sysents for "machine"

In [63]:
wn.synsets('machine')

LookupError: 
**********************************************************************
  Resource [93mwordnet[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('wordnet')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/wordnet[0m

  Searched in:
    - 'C:\\Users\\mikol/nltk_data'
    - 'C:\\Users\\mikol\\AppData\\Local\\Programs\\Python\\Python37\\nltk_data'
    - 'C:\\Users\\mikol\\AppData\\Local\\Programs\\Python\\Python37\\share\\nltk_data'
    - 'C:\\Users\\mikol\\AppData\\Local\\Programs\\Python\\Python37\\lib\\nltk_data'
    - 'C:\\Users\\mikol\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


2.3.3) Display all definitions (.definition()) for synsets (machine)

In [ ]:
#TODO
print(wn.synsets('machine').definition())

2.3.4) For each synset (machine), display its hypernym (a word with a broad meaning constituting a category into which words with more specific meanings fall; a superordinate. For example, colour is a hypernym of red).

In [ ]:
#TODO
print(wn.synets('machine').hypernym())

See: http://www.nltk.org/howto/wordnet.html
for more examples